In [1]:
from UTILITY_quickstart import *

In [2]:
#Start from golden lattice
tao = initializeTao(
    loadDefaultLatticeTF = False
)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Base Tao lattice
WARNING! No beam file is specified!
Number of macro particles defined by input file


In [3]:
getMatrix(tao, "BEGBC20", "MFFF", print = True);

-1.208438,-4.961819,-0.000000,-0.000000,0.000000,-0.000000
-0.084842,-1.175874,-0.000000,-0.000000,0.000000,-0.000000
-0.000000,-0.000000,-0.638379,6.515289,0.000000,-0.000000
-0.000000,-0.000000,-0.094469,-0.602321,0.000000,-0.000000
0.000000,0.000000,0.000000,0.000000,1.000000,-0.007000
0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [4]:
#Load a default configuration
#importedDefaultSettings = loadConfig("setLattice_configs/2024-10-22_oneBunch_baseline.yml")
importedDefaultSettings = loadConfig("setLattice_configs/defaults.yml")

#Initialize tao object
tao = initializeTao(
    #inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
    #numMacroParticles = 1e4,
    #csrTF = False
)

#Set lattice to the loaded config
setLattice(tao, **importedDefaultSettings)

getMatrix(tao, "BEGBC20", "MFFF", print = True);

launchTwissCorrection(tao)

baseMatrix = getMatrix(tao, "BEGBC20", "MFFF", print = True);

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using setLattice_configs/defaults.yml
WARNING! No beam file is specified!
Number of macro particles defined by input file


-1.208675,-4.962570,-0.000000,-0.000000,0.000000,0.000012
-0.084965,-1.176201,0.000000,0.000000,0.000000,-0.000001
-0.000000,-0.000000,-0.637675,6.526881,0.000000,-0.000000
-0.000000,-0.000000,-0.094432,-0.601645,0.000000,-0.000000
-0.000003,-0.000021,0.000000,-0.000000,1.000000,-0.007001
0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


No evalElement provided. Assuming golden lattice PR10571
Optimization Results:
Optimal Parameters: [0.13755652 0.953558   0.40561699 2.20425982]
Objective Function Value at Optimal Parameters: 3.3948865246987365e-09
Number of Iterations: 235
Converged: True


-1.208675,-4.962570,-0.000000,-0.000000,0.000000,0.000012
-0.084965,-1.176201,0.000000,0.000000,0.000000,-0.000001
-0.000000,-0.000000,-0.637675,6.526881,0.000000,-0.000000
-0.000000,-0.000000,-0.094432,-0.601645,0.000000,-0.000000
-0.000003,-0.000021,0.000000,-0.000000,1.000000,-0.007001
0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [5]:
setLattice(tao, **(importedDefaultSettings | {'Q3EkG': 110}))

newMatrix = getMatrix(tao, "BEGBC20", "MFFF", print = True);

-1.310429,-5.082830,-0.000000,-0.000000,0.000000,0.007158
-0.128425,-1.261239,0.000000,0.000000,0.000000,0.000397
-0.000000,-0.000000,-0.692087,5.709292,0.000000,-0.000000
0.000000,-0.000000,-0.095684,-0.655575,0.000000,-0.000000
-0.000399,-0.007012,0.000000,0.000000,1.000000,-0.007487
0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [6]:
displayMatrix(  newMatrix - baseMatrix ) 

-0.101754,-0.120260,-0.000000,-0.000000,0.000000,0.007145
-0.043460,-0.085038,-0.000000,-0.000000,0.000000,0.000398
0.000000,0.000000,-0.054411,-0.817590,0.000000,0.000000
0.000000,0.000000,-0.001252,-0.053930,0.000000,0.000000
-0.000397,-0.006991,-0.000000,0.000000,0.000000,-0.000486
0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
def getNewMatrix(params):
    q1, q2, q3, q4, q5, q6 = params
    setLattice(tao, **(importedDefaultSettings | {
        'Q1EkG': q1,
        'Q2EkG': q2,
        'Q3EkG': q3,
        'Q4EkG': q4,
        'Q5EkG': q5,
        'Q6EkG': q6,

    }))

    return( getMatrix(tao, "BEGBC20", "MFFF") )

In [8]:
baselineParams = [ importedDefaultSettings[ele] for ele in ['Q1EkG', 'Q2EkG', 'Q3EkG', 'Q4EkG', 'Q5EkG', 'Q6EkG'] ]
baselineParams

[161.311, -154.229, 110.217, 132.268, -23.373, -142.271]

In [9]:
getNewMatrix( baselineParams )

array([[-1.20867456e+00, -4.96256976e+00, -1.33718758e-17,
        -1.10207470e-16,  0.00000000e+00,  1.23649164e-05],
       [-8.49649366e-02, -1.17620117e+00, -1.90000751e-18,
        -2.33770173e-17,  0.00000000e+00, -1.24861596e-06],
       [-2.38694683e-16, -1.12476605e-15, -6.37675400e-01,
         6.52688129e+00,  0.00000000e+00, -2.74050514e-17],
       [-2.92697655e-18, -1.41951754e-17, -9.44318686e-02,
        -6.01645293e-01,  0.00000000e+00, -8.06887982e-19],
       [-2.55975468e-06, -2.07399729e-05,  1.46759644e-19,
         8.85450066e-18,  1.00000000e+00, -7.00081917e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [10]:
#The objective is to create a transport matrix that is the same as the baseline, except with a larger dispersion term

def objective(params):
    newMatrix = getNewMatrix(params)

    targetMatrix = baseMatrix + [[0,0,0,0,0,0.01],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

    errorMatrix = np.array( newMatrix - targetMatrix )

    error = np.sum(errorMatrix ** 2) 

    #print(error)
    
    return error
    

    


In [11]:
from scipy.optimize import minimize

# Perform optimization using Nelder-Mead
result = minimize(
    objective, 
    baselineParams, #Starting point
    method='Nelder-Mead',
)

                      
print("Optimization Results:")
print(f"Optimal Parameters: {result.x}")
print(f"Objective Function Value at Optimal Parameters: {result.fun}")
print(f"Number of Iterations: {result.nit}")
print(f"Converged: {result.success}")

Optimization Results:
Optimal Parameters: [ 162.383757   -154.1009491   109.57661266  134.41302407  -23.66569387
 -142.68966868]
Objective Function Value at Optimal Parameters: 4.956145439395259e-05
Number of Iterations: 541
Converged: True


In [12]:
proposedParams = result.x

In [13]:
proposedParams - baselineParams

array([ 1.072757  ,  0.1280509 , -0.64038734,  2.14502407, -0.29269387,
       -0.41866868])

In [14]:
(proposedParams - baselineParams) / baselineParams

array([ 0.00665024, -0.00083026, -0.00581024,  0.01621726,  0.01252273,
        0.00294275])

### X Dispersion prime

In [15]:
def objective(params):
    newMatrix = getNewMatrix(params)

    targetMatrix = baseMatrix + [[0,0,0,0,0,0],[0,0,0,0,0,0.01],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

    errorMatrix = np.array( newMatrix - targetMatrix )

    error = np.sum(errorMatrix ** 2) 

    #print(error)
    
    return error

In [ ]:
# Perform optimization using Nelder-Mead
result = minimize(
    objective, 
    baselineParams, #Starting point
    method='Nelder-Mead',
)

                      
print("Optimization Results:")
print(f"Optimal Parameters: {result.x}")
print(f"Objective Function Value at Optimal Parameters: {result.fun}")
print(f"Number of Iterations: {result.nit}")
print(f"Converged: {result.success}")

In [ ]:
proposedParams = result.x

In [ ]:
proposedParams - baselineParams